In [1]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import pickle
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
import os, sys

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [2]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines =False)
data['publish_date'] = pd.to_datetime(data['publish_date'].astype(str), format = '%Y%m%d')
data['publish_date'] = pd.DatetimeIndex(data['publish_date']).year

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [4]:
data_words = list(sent_to_words(data['headline_text']))
data_words_nostops = remove_stopwords(data_words)
data_words_nostops = pd.Series(data_words_nostops)
data_lemmatized = data_words_nostops.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [19]:
with open('./pickle_data/data_lemmatized.pickle', 'wb') as f:
    pickle.dump(data_lemmatized, f, pickle.HIGHEST_PROTOCOL)

In [7]:
stop_words.extend(['interview', 'weather', 'abc', 'australia', 'australian','queensland', 'australias','sydney', 'nz', 'plan','get','back','new'])

In [8]:
stop_words.extend(['urge','group','council','say','report','two','qld','call','nsw','may','take','charge','seek','police','make','tas'])

In [9]:
stop_words.extend(['meet','win','talk', 'govt','find','man','set','wa','continue'])

In [10]:
data_lemmatized = remove_stopwords(data_lemmatized)

In [11]:
temp = data
temp['lemmatize'] = data_lemmatized
for i in range(15):
    globals()['trend{}'.format(i+2003)] = temp.loc[temp.publish_date == i+2003]

trend_list = [trend2003, trend2004, trend2005, trend2006, trend2007, trend2008, trend2009, trend2010, trend2011, trend2012, trend2013, trend2014, trend2015, trend2016, trend2017]

In [24]:
for i, year in enumerate(trend_list):    
    num_topics = 40
    lemmatized = list(year['lemmatize'])
    id2word = corpora.Dictionary(lemmatized)
#     id2word.filter_extremes(no_below=5, no_above=0.3, keep_n=10000)
    texts = lemmatized
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    with open("./pickle_data/result_"+str(i+2003)+".pickle", 'wb') as f:
        pickle.dump(lda_model.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=True), f, pickle.HIGHEST_PROTOCOL)

    pyLDAvis.enable_notebook()
    prepared_data  = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    pyLDAvis.save_html(prepared_data, './result/LDA_result'+str(i+2003)+'.html')   

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [16]:
lemmatized = list(trend2010['lemmatize'])
id2word = corpora.Dictionary(lemmatized)
id2word.filter_extremes(no_below=5, no_above=0.3, keep_n=10000)
texts = lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
lda_model.show_topics(num_topics=30, num_words=10, log=False, formatted=True)

list

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19    -0.261969 -0.031945       1        1  4.216392
28    -0.259863 -0.055154       2        1  4.159419
20     0.061027 -0.118935       3        1  4.083325
4      0.072755 -0.214912       4        1  3.870208
26    -0.167740 -0.000232       5        1  3.834130
14     0.038163  0.039734       6        1  3.720423
27     0.077483 -0.195865       7        1  3.692938
7      0.050427 -0.164290       8        1  3.668810
15     0.046807  0.035740       9        1  3.645311
1     -0.038168  0.023033      10        1  3.614103
2      0.034931  0.071701      11        1  3.490355
6      0.037338  0.052114      12        1  3.456371
9      0.038026  0.071508      13        1  3.452439
11     0.020079  0.042852      14        1  3.434582
16     0.002665 -0.049692      15        1  3.304619
13    -0.155994 -0.018629      16        1  3.296787
8      0.046623  0.154808      17        1  3.270447
24     0.023008  0.046789      18        1  3.216359
5      0.039941  0.015522      19        1  3.187201
3      0.037934  0.029753      20        1  3.128191
29     0.059287 -0.121677      21        1  3.082557
10     0.053289  0.002048      22        1  3.032179
17     0.026411  0.031356      23        1  2.946375
25     0.037541  0.049085      24        1  2.850542
0      0.052467  0.033818      25        1  2.784239
18    -0.051995  0.021624      26        1  2.781086
21     0.007696  0.035045      27        1  2.717380
23     0.027235  0.041309      28        1  2.716451
22     0.006254  0.118559      29        1  2.688107
12     0.038342  0.054931      30        1  2.658680, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
39    Default  1391.000000       flood  1391.000000  30.0000  30.0000
505   Default  1324.000000       court  1324.000000  29.0000  29.0000
563   Default  1162.000000        fire  1162.000000  28.0000  28.0000
565   Default  1135.000000       water  1135.000000  27.0000  27.0000
5     Default  1126.000000       crash  1126.000000  26.0000  26.0000
371   Default  1089.000000        mine  1089.000000  25.0000  25.0000
1027  Default  1070.000000        face  1070.000000  24.0000  24.0000
607   Default  1084.000000        warn  1084.000000  23.0000  23.0000
688   Default   992.000000        jail   992.000000  22.0000  22.0000
10    Default  1083.000000        kill  1083.000000  21.0000  21.0000
419   Default   918.000000       death   918.000000  20.0000  20.0000
810   Default   937.000000       house   937.000000  19.0000  19.0000
300   Default   967.000000      attack   967.000000  18.0000  18.0000
752   Default   924.000000      murder   924.000000  17.0000  17.0000
541   Default   794.000000         day   794.000000  16.0000  16.0000
386   Default   868.000000          us   868.000000  15.0000  15.0000
343   Default   816.000000        miss   816.000000  14.0000  14.0000
175   Default   856.000000        home   856.000000  13.0000  13.0000
834   Default   744.000000        fear   744.000000  12.0000  12.0000
101   Default   817.000000         hit   817.000000  11.0000  11.0000
141   Default   736.000000      school   736.000000  10.0000  10.0000
1219  Default   768.000000      health   768.000000   9.0000   9.0000
2056  Default   754.000000       green   754.000000   8.0000   8.0000
766   Default   701.000000       coast   701.000000   7.0000   7.0000
1200  Default   712.000000          go   712.000000   6.0000   6.0000
738   Default   716.000000      change   716.000000   5.0000   5.0000
228   Default   730.000000        fund   730.000000   4.0000   4.0000
179   Default   662.000000        open   662.000000   3.0000   3.0000
87    Default   731.000000         car   731.000000   2.0000   2.0000
62    Default   714.000000       boost   714.000000   1.0000   1.0000
...       ...        